# 语言模型的评价

perplexity

$\begin{aligned} P P(W) &=P\left(w_{1} w_{2} \ldots w_{N}\right)^{-\frac{1}{N}} \\ &=\sqrt{\frac{1}{P\left(w_{1} w_{2} \ldots w_{N}\right)}} \\ \operatorname{PP}(W) &=\sqrt[n]{\prod_{i=1}^{N} \frac{1}{P\left(w_{i} | w_{1} \ldots w_{i-1}\right)}} \end{aligned}$


-1是因为数字很小


1/N是一段话长N个单词

越低越好

$J=-\frac{1}{T} \sum_{t=1}^{T} \sum_{j=1}^{|V|} y_{t, j} \log \hat{y}_{t, j}$

perplexity=2^J

# 第三课 语言模型和torchtext(专门为NLP开发的库）

褚则伟 zeweichu@gmail.com

学习目标
- 学习语言模型，以及如何训练一个语言模型
- 学习torchtext的基本使用方法
    - 构建 vocabulary
    - word to inde 和 index to word
- 学习torch.nn的一些基本模型
    - Linear
    - RNN
    - LSTM
    - GRU
- RNN的训练技巧
    - Gradient Clipping
- 如何保存和读取模型

In [1]:
import torchtext
from torchtext.vocab import Vectors
import torch
import numpy as np
import random

USE_CUDA = torch.cuda.is_available()

# 为了保证实验结果可以复现，我们经常会把各种random seed固定在某一个值
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)

BATCH_SIZE = 32
EMBEDDING_SIZE = 100
MAX_VOCAB_SIZE = 50000

In [2]:
import transformers

I0207 16:06:25.186644 140020917724928 file_utils.py:38] PyTorch version 1.1.0 available.


- 我们会继续使用上次的text8作为我们的训练，验证和测试数据
- TorchText的一个重要概念是`Field`，它决定了你的数据会如何被处理。我们使用`TEXT`这个field来处理文本数据。我们的`TEXT` field有`lower=True`这个参数，所以所有的单词都会被lowercase。
- torchtext提供了LanguageModelingDataset这个class来帮助我们处理语言模型数据集。
- `build_vocab`可以根据我们提供的训练数据集来创建最高频单词的单词表，`max_size`帮助我们限定单词总量。
- BPTTIterator可以连续地得到连贯的句子，BPTT的全程是back propagation through time。


In [3]:
#用torch.text完成把文本读进来的问题
#第一步：定义Field,相当于告诉程序，我输入的是一些什么
#因为全都是单词，所以可以lowercase
TEXT = torchtext.data.Field(lower=True)
#torch中的LanguageModelingDataset是专门给文本文件写的Dataset,#path="."就是全在当前文件夹下
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(path=".", 
    train="text8.train.txt", validation="text8.dev.txt", test="text8.test.txt", text_field=TEXT)
#pytorch写好了如何构建单词表
TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)
print("vocabulary size: {}".format(len(TEXT.vocab)))

vocabulary size: 50002


In [4]:
TEXT.vocab.itos[:10]#unk表示不常见或者不认识的字词，<pad>是为了防止句子太短

['<unk>', '<pad>', 'the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero']

In [5]:
TEXT.vocab.stoi["<unk>"]#字典

0

In [6]:
TEXT.vocab.stoi["apple"]#字典

1273

In [7]:
VOCAB_SIZE = len(TEXT.vocab)
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=BATCH_SIZE, device = torch.device("cuda:3"), bptt_len=32, repeat=False, shuffle=True)
#bptt_len是指：根据时间步的反向传播BPTT的长度
device = torch.device("cuda:3")

In [8]:
it = iter(train_iter)
batch = next(it)
print(".text")#当前的文本
print(" ".join([TEXT.vocab.itos[i] for i in batch.text[:,1].data]))
print("target")#需要预测的文本
print(" ".join([TEXT.vocab.itos[i] for i in batch.target[:,1].data]))
#发现只差了一个位置

.text
had dropped to just three zero zero zero k it was then cool enough to allow the nuclei to capture electrons this process is called recombination during which the first neutral atoms
target
dropped to just three zero zero zero k it was then cool enough to allow the nuclei to capture electrons this process is called recombination during which the first neutral atoms took


In [9]:
for i in range(5):
    batch = next(it)
    print(i,":")
    print(" ".join([TEXT.vocab.itos[i] for i in batch.text[:,2].data]))
    print()
    print(" ".join([TEXT.vocab.itos[i] for i in batch.target[:,2].data]))

0 :
at first contact with the japanese people was friendly and both were equals in a trade relationship however eventually the japanese started to dominate the relationship and soon established large settlements on

first contact with the japanese people was friendly and both were equals in a trade relationship however eventually the japanese started to dominate the relationship and soon established large settlements on the
1 :
the outskirts of ainu territory as the japanese moved north and took control over their traditional lands the ainu often gave up without resistance but there was occasional resistance as exemplified in

outskirts of ainu territory as the japanese moved north and took control over their traditional lands the ainu often gave up without resistance but there was occasional resistance as exemplified in wars
2 :
wars in one four five seven one six six nine and one seven eight nine all of which were lost by the ainu japanese policies became increasingly aimed at assimi

In [10]:
import torch
import torch.nn as nn


class RNNModel(nn.Module):
    """ 一个简单的循环神经网络"""
    #rnn_type是指循环神经网络的类型，ntoken是指vocab_size
    #nlayers是指可以一层套一层
    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        ''' 该模型包含以下几层:
            - 词嵌入层
            - 一个循环神经网络层(RNN, LSTM, GRU)
            - 一个线性层，从hidden state到输出单词表
            - 一个dropout层，用来做regularization
        '''
        super(RNNModel, self).__init__()#这段背下来就行了
        ###########################
        #pytorch自己有LSTM
        ##########################
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        ''' Forward pass:
            - word embedding
            - 输入循环神经网络
            - 一个线性层从hidden state转化为输出单词表
        '''
        #习惯把size写下来text:seq_length*batch_size,默认batch_size是在后面的
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        #output:seq_length*batch_size*hidden_size
        #hidden有两个，因为LSTM传递了两个:(1*batch_size*hidden_size，1*batch_size*hidden_size)
        #用1乘是因为只有一层，其实可以有好多层好多方向
        
        output = self.drop(output)
        
        #decode:(seq_length*batch_size)*vocab_size
        #这里其实可以加激活层，但是一般都不加，相当于惯例
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        #因为decode是线性变换（其实应该叫linear），只能两维？？所以要先view?  视频50:58
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz, requires_grad=True):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            #########################
            #使用weight.new_zeros()创建纯零矩阵，因为不知道数据在GPU还是CPU上，所以创建一个相同类型的
            ##################
            return (weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad),
                    weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad))
        else:
            return weight.new_zeros((self.nlayers, bsz, self.nhid), requires_grad=requires_grad)

# 如果使用模型读取的方法，则可直接去运行模型读取部分的kernal，不必初始化

# 但如果想查看模型的pp等，还是要初始化并定于检查函数等
初始化一个模型

In [11]:
model = RNNModel("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
if USE_CUDA:
    model = model.cuda(device = torch.device("cuda:3"))

In [22]:
#查看一下模型在哪
next(model.parameters())

Parameter containing:
tensor([[-0.0148, -0.0025, -0.0085,  ..., -0.0337,  0.0600, -0.0214],
        [-0.0485, -0.1032, -0.0134,  ..., -0.1146,  0.1165, -0.0539],
        [ 0.2117, -0.2793, -0.3717,  ..., -0.4380,  0.0778, -0.4988],
        ...,
        [-0.2929,  0.1851, -0.0309,  ...,  0.0359, -0.0244, -0.1295],
        [ 0.0136, -0.0324, -0.0006,  ...,  0.0716, -0.0505, -0.1268],
        [-0.1441,  0.1106,  0.0614,  ...,  0.0405, -0.0568, -0.0181]],
       device='cuda:3', requires_grad=True)

# 训练模型，所有模型训练都是大同小异

- 我们首先定义评估模型的代码。
- 模型的评估和模型的训练逻辑基本相同，唯一的区别是我们只需要forward pass，不需要backward pass，还有进来的时候首先model.eval()，最后改回model.train()

In [23]:
def evaluate(model, data):
    model.eval()
    total_loss = 0.
    it = iter(data)
    total_count = 0.
    with torch.no_grad():#因为在评价，所以要确保没有梯度
        hidden = model.init_hidden(BATCH_SIZE, requires_grad=False)
        for i, batch in enumerate(it):
            data, target = batch.text, batch.target
            if USE_CUDA:
                data, target = data.cuda(device = torch.device("cuda:3")), target.cuda(device = torch.device("cuda:3"))
            hidden = repackage_hidden(hidden)
            with torch.no_grad():
                output, hidden = model(data, hidden)
            loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1))
            total_count += np.multiply(*data.size())
            total_loss += loss.item()*np.multiply(*data.size())
            
    loss = total_loss / total_count
    model.train()
    return loss


我们需要定义下面的一个function，帮助我们把一个hidden state和计算图之前的历史分离。

In [24]:
# Remove this part
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

定义loss function和optimizer


In [25]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler有很多玩法，比如三次loss不下降就把lr砍半等等
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)#0.5指减半

# 训练模型：
### 如果使用读取模型的方法，则下面不需要运行
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- gradient clipping，防止梯度爆炸
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，以及在验证集上做模型的评估

In [26]:
import copy
GRAD_CLIP = 1.
NUM_EPOCHS = 2

val_losses = []
for epoch in range(NUM_EPOCHS):
    model.train()
    it = iter(train_iter)
    hidden = model.init_hidden(BATCH_SIZE)
    for i, batch in enumerate(it):
        data, target = batch.text, batch.target
        if USE_CUDA:
            data, target = data.cuda(device = torch.device("cuda:3")), target.cuda(device = torch.device("cuda:3"))
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        optimizer.step()
        if i % 1000 == 0:
            print("epoch", epoch, "iter", i, "loss", loss.item())
    
        if i % 10000 == 0:
            val_loss = evaluate(model, val_iter)
            ###########################
            #保存模型
            #############
            #但一般不会每个模型都保存，根据val_loss，把最好的模型存下来
            if len(val_losses) == 0 or val_loss < min(val_losses):
                print("best model, val loss: ", val_loss)
                torch.save(model.state_dict(), "lm-best.th")
            else:
                ################################
                #如果loss不下降，需要更精细的lr，pytorch提供了scheduler
                ################################
                scheduler.step()#指lr下降一些，如果参数是0.5就下降50%
                
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            val_losses.append(val_loss)

epoch 0 iter 0 loss 6.193421840667725
best model, val loss:  5.618237644952179
epoch 0 iter 1000 loss 6.182553291320801
epoch 0 iter 2000 loss 6.2674174308776855
epoch 0 iter 3000 loss 6.141019344329834
epoch 0 iter 4000 loss 5.621214389801025
epoch 0 iter 5000 loss 6.087187767028809
epoch 0 iter 6000 loss 6.171429634094238
epoch 0 iter 7000 loss 5.931452751159668
epoch 0 iter 8000 loss 6.111051559448242
epoch 0 iter 9000 loss 5.865108966827393
epoch 0 iter 10000 loss 5.980441570281982
best model, val loss:  5.586545678821775
epoch 0 iter 11000 loss 6.097061634063721
epoch 0 iter 12000 loss 6.233752727508545
epoch 0 iter 13000 loss 5.877536773681641
epoch 0 iter 14000 loss 5.826540470123291
epoch 1 iter 0 loss 6.1389923095703125
best model, val loss:  5.55586278224108
epoch 1 iter 1000 loss 6.06961727142334
epoch 1 iter 2000 loss 6.127335548400879
epoch 1 iter 3000 loss 6.035078525543213
epoch 1 iter 4000 loss 5.511624813079834
epoch 1 iter 5000 loss 6.100916385650635
epoch 1 iter 6000

模型所有的参数

In [17]:
model.state_dict()

OrderedDict([('encoder.weight',
              tensor([[-0.0148, -0.0025, -0.0085,  ..., -0.0337,  0.0600, -0.0214],
                      [-0.0485, -0.1032, -0.0134,  ..., -0.1146,  0.1165, -0.0539],
                      [ 0.2117, -0.2793, -0.3717,  ..., -0.4380,  0.0778, -0.4988],
                      ...,
                      [-0.2929,  0.1851, -0.0309,  ...,  0.0359, -0.0244, -0.1295],
                      [ 0.0136, -0.0324, -0.0006,  ...,  0.0716, -0.0505, -0.1268],
                      [-0.1441,  0.1106,  0.0614,  ...,  0.0405, -0.0568, -0.0181]],
                     device='cuda:3')),
             ('rnn.weight_ih_l0',
              tensor([[-0.0071, -0.1687,  0.1917,  ...,  0.1233, -0.0971, -0.1381],
                      [-0.2608,  0.5052,  0.5019,  ...,  0.3304,  0.2009,  0.3287],
                      [ 0.8329,  0.2731,  0.3885,  ...,  0.0246,  0.4055,  0.9056],
                      ...,
                      [ 0.0065,  0.5008,  0.1155,  ...,  0.2647, -0.3502,  0.0219],

# Load模型
- 先新定义一个模型
- 用model.load_state_dict()

In [18]:
#Load模型
best_model = RNNModel("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
if USE_CUDA:
    best_model = best_model.cuda(device = torch.device("cuda:3"))
best_model.load_state_dict(torch.load("lm-best.th"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

### 使用最好的模型在valid数据上计算perplexity

In [19]:
#好的结果是140,170等
val_loss = evaluate(best_model, val_iter)
print("perplexity: ", np.exp(val_loss))

perplexity:  287.3899864294951


### 使用最好的模型在测试数据上计算perplexity

In [20]:
test_loss = evaluate(best_model, test_iter)
print("perplexity: ", np.exp(test_loss))

perplexity:  340.36256797664157


# 使用训练好的模型生成一些句子
- 这里只用了1个hidden_state
- 把hidden_state传入best_model里面
- 用multinomial来sample一下（也可以用argmax，预测最大的单词概率）

In [21]:
hidden = best_model.init_hidden(1)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
input = torch.randint(VOCAB_SIZE, (1, 1), dtype=torch.long).to(device)
words = []
for i in range(100):
    #run forward pass
    output, hidden = best_model(input, hidden)
    #logists exp
    word_weights = output.squeeze().exp().cpu()
    #multunomial sampling 拿到一个单词，也可以argmax，但是argmax往往不是最好
    #用argmax或者beam search的话每次生成的理论上是同一句话
    #但是如果翻译之类的任务会喜欢用argmax或者beam search，因为比较diterminastic
    word_idx = torch.multinomial(word_weights, 1)[0]
    #fill in the current predicted word to the current input
    #接下来再用这个input去预测下一个input
    input.fill_(word_idx)
    word = TEXT.vocab.itos[word_idx]
    words.append(word)
print(" ".join(words))

explosives are goss the water press that franks within their culture were the people made the politics of game at potency and hospital this commander was depicted to some so you have collectively become the body potential member of the open association organization in turn of this would be divided out for two proudly rise the flop browser list of user and substitution alone qc or processing which considerations also designation generated in line and a two this video rewarding effects that sides that precludes overlapping regions smelling but anxiety within a large ratio of the maple duck <unk> there
